In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anisa116","key":"337ca6220b1c8a3262708098614ecb4f"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download zeyadkhalid/faceshape-processed

 71% 57.0M/79.8M [00:00<00:00, 93.5MB/s]
100% 79.8M/79.8M [00:00<00:00, 114MB/s] 


In [ ]:
! unzip faceshape-processed

Archive:  faceshape-processed.zip
  inflating: dataset/test/Heart/Heart(0).jpg  
  inflating: dataset/test/Heart/Heart(1).jpg  
  inflating: dataset/test/Heart/Heart(10).jpg  
  inflating: dataset/test/Heart/Heart(100).jpg  
  inflating: dataset/test/Heart/Heart(101).jpg  
  inflating: dataset/test/Heart/Heart(102).jpg  
  inflating: dataset/test/Heart/Heart(103).jpg  
  inflating: dataset/test/Heart/Heart(104).jpg  
  inflating: dataset/test/Heart/Heart(105).jpg  
  inflating: dataset/test/Heart/Heart(106).jpg  
  inflating: dataset/test/Heart/Heart(107).jpg  
  inflating: dataset/test/Heart/Heart(108).jpg  
  inflating: dataset/test/Heart/Heart(109).jpg  
  inflating: dataset/test/Heart/Heart(11).jpg  
  inflating: dataset/test/Heart/Heart(110).jpg  
  inflating: dataset/test/Heart/Heart(111).jpg  
  inflating: dataset/test/Heart/Heart(112).jpg  
  inflating: dataset/test/Heart/Heart(113).jpg  
  inflating: dataset/test/Heart/Heart(114).jpg  
  inflating: dataset/test/Heart/Heart(115

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# Test 1

# Set the image dimensions and batch size
img_width, img_height = 190, 250
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip = True,
    brightness_range=(0.8,1.2)
    )

test_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip = True,
    brightness_range=(0.8,1.2)
    )


training_set = train_datagen.flow_from_directory(
    '/content/dataset/train',
    target_size = (img_width, img_height),
    batch_size = 64,
    color_mode = 'grayscale',
    shuffle = True,
    class_mode = 'categorical'
    )

test_set = test_datagen.flow_from_directory(
    '/content/dataset/test',
    target_size = (img_width, img_height),
    batch_size = 64,
    color_mode = 'grayscale',
    shuffle=True,
    class_mode = 'categorical'
    )

NameError: ignored

In [ ]:
# Test 1

# Define the CNN model architecture
model = Sequential([
    Conv2D(8, (7, 7), activation='selu', padding='valid', input_shape=(img_width, img_height, 1)),
    Conv2D(8, (5, 5), activation='selu', padding='valid'),

    MaxPooling2D(pool_size=(5, 5), padding="valid", strides=(2, 2)),
    MaxPooling2D(pool_size=(3, 3), padding="valid", strides=(2, 2)),

    Flatten(),
    Dense(1024, activation='selu'),
    Dense(5, activation='softmax')  # Assuming 5 face shape classes
])

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.summary()

In [ ]:
X_train, y_train = next(training_set)
X_test, y_test = next(test_set)

In [ ]:
# from PIL import ImageFile
# ImageFile.LOAD_TRUNCATED_IMAGES = True

# Train the model
history = model.fit(
    training_set,
    steps_per_epoch=batch_size,
    epochs=100,
    validation_data=test_set,
    shuffle=True
    )

In [ ]:
import matplotlib.pyplot as plt

# Plot the chart for accuracy and loss on both training and validation
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# Save the model
model.save('face_shape_classifier.h5')

In [ ]:
from google.colab import auth


In [ ]:
auth.authenticate_user()

In [ ]:
CLOUD_PROJECT = 'capstone-c23ps466-1'

In [ ]:
BUCKET = 'gs://' + CLOUD_PROJECT + '-tf2-models'

In [ ]:
!gsutil mb $BUCKET
print(BUCKET)

In [ ]:
!gcloud config set project $CLOUD_PROJECT

In [ ]:
!gcloud ai-platform models create frame_wiz

In [ ]:
VERSION = 'v2'

In [ ]:
MODEL_DIR = BUCKET + '/frame_wiz'

In [ ]:
model.save(BUCKET + '/frame_wiz', save_format='tf')

In [ ]:
!gcloud ai-platform versions create $VERSION \
  --model frame_wiz \
  --origin $MODEL_DIR \
  --runtime-version=2.1 \
  --framework='tensorflow' \
  --python-version=3.7

In [ ]:
!gcloud config set ai_platform/region us-central1

In [ ]:
import googleapiclient.discovery

def predict_json(project, model, instances, version=None):

    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [ ]:
!test_predictions = predict_json(CLOUD_PROJECT, MODEL, test_images[:2].tolist())